# **Addons**

Addons is a feature of dicfg that can be added to the config dictionary keys via ```@addon_type{addon_name}``` syntax.
Currently there are these types of addons:
- validator (@validate(...))
- updater (@update(...))
- templates (@validate(...))

Below these types are discussed in more detail.


In [5]:
from pprint import pprint

from dicfg import ConfigReader

## Validators: use validators with **@validate(validator-name)** in config files

Dicfg has the following builtin validators: 
 - **required**: using this validator throws an error if the value is empty, useful when a value is required to be set by a user.
 - **depreciated**: using this validator throws an error if the values is not empty, useful when a name has changed and should'nt be used anymore
 - **object**: using this validator throws an error if the object is not defined with "*object", or if object can not be loaded, or if arguments are not valid arguments. 

In [27]:
print("Main config:\n")
!cat ./configs/config_with_validators.yml
config_reader = ConfigReader(name="myconfig", main_config_path="./configs/config_with_validators.yml")
user_config = {'myconfig': {'default': {'config_int2': 3}}}
print("\n"+"Output config:\n")
pprint(config_reader.read(user_config), sort_dicts=False)  

Main config:

default:
    config_int2@validator(required): 
    config_int@validator(depreciated): 
    object:
      "*object": "io.StringIO"
    config_string: "fire"
    config_list: [1, 2, 3]
    config_none: None
    config_dict:
        sub_config: "water"


        
Output config:

{'default': {'config_int2': 3,
             'config_int': None,
             'object': {'*object': 'io.StringIO'},
             'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'}}}


## Updaters: use updates with **@update(updater-name)** in config files

Dicfg has the following builtin validators: 
 - **replace**: using this updater replaces a list or dict completely
 - **merge**: using this updater merges a list or dict with the additional values

**Note**: Default behaviour or a dict is merge and the default behaviour of a list is replace

In [28]:
print("Main config:\n")
!cat ./configs/config_with_updaters.yml
config_reader = ConfigReader(name="myconfig", main_config_path="./configs/config_with_updaters.yml")
user_config = {'myconfig': {'default': {'config_list': [20], 'config_dict': {'new': 'new'}}}}
print("\n"+"Output config:\n")
pprint(config_reader.read(user_config), sort_dicts=False)  

Main config:

default:
    config_string: "fire"
    config_list@updater(merge): [1, 2, 3]
    config_none: None
    config_dict@updater(replace):
        sub_config: "water"


        
Output config:

{'default': {'config_string': 'fire',
             'config_list': [1, 2, 3, 20],
             'config_none': 'None',
             'config_dict': {'new': 'new'}}}


## Templates: use templates with **@template(template-name)** in config files

- **verbose**: Enables verbose logging.
- **debug**: Enables debug logging.
- **build**: Initiates the build process.
- **dontbuild**: Disables the build process.
- **cwd**: Returns the current working directory.
- **systeminfo**: Provides detailed system information.
- **datetimenow**: Outputs the current date and time.
- **uuid4**: Generates a random UUID (version 4).
- **sqlwrite**: Writes data to an SQLite database.

## Modifiers: use modifiers with **@modifier(modifier-name)** in config files

Modifiers are are able to modify a value

- **include**: Loads and processes external file content based on its format.
- **command**: Executes a shell command and returns its output.
- **slugify**: Converts text into a URL-friendly slug.
- **date**: Returns the current date formatted as specified.
- **encodebase64**: Encodes text into Base64.
- **decodebase64**: Decodes a Base64-encoded string back to text.
- **uuid5**: Generates a version 5 UUID from a given namespace and name.
- **math**: Safely evaluates a mathematical expression.
- **env**: Retrieves the value of an environment variable.
- **sqlread**: Executes an SQL query on a SQLite database and formats the results.

### Note: 
**modifiers dont work with interpolation**. This is because modifiers are applied before building the config. If you need that functionalitiy, please use *object: and modify when building  


In [2]:
from pprint import pprint

from dicfg import ConfigReader
print("Main config:\n")
!cat "./configs/config_with_addons.yml"
config_reader = ConfigReader(name="myconfig", main_config_path="./configs/config_with_addons.yml")
print("\n"+"Output config:\n")
pprint(config_reader.read(), sort_dicts=False)  

Main config:

default:
    config_string: "fire"
    config_list: [1, 2, 3]
    config_none: None
    config_dict:
        sub_config: "water"
    mydate@template(datetimenow):
    mynumber%math: 2+3*8
    myuuid#uuid4:

        
Output config:

{'default': {'config_string': 'fire',
             'config_list': [1, 2, 3],
             'config_none': 'None',
             'config_dict': {'sub_config': 'water'},
             'mydate': '2025-02-14 15:04:33.745174',
             'mynumber': 26,
             'myuuid': '99946e68-e68a-4691-b30b-c1ad439de35e'}}
